分配関数の計算を避けたい．

ネガティブサンプルに対する勾配の計算は不要．

スコア関数自体が対数尤度の勾配の形なので，ランジュバンダイナミクスによる勾配の計算は不要．

# SBM

EBMでは，分配関数の計算のために，negative samplingをしていた．

結局はそこがボトルネックとなっており，ランジュバンダイナミクスなどの手法で，サンプリングしていた．

そこを改善したいのがSBM．

まず，EBMの尤度(厳密には負の対数尤度)の勾配を確認する．

$$
\nabla_{\theta}\mathbb{E}_{\boldsymbol{x} \sim p_{data}}[-\log p_{\theta}(\boldsymbol{x})] = \mathbb{E}_{\boldsymbol{x} \sim p_{data}}[\nabla_{\theta} E_{\theta}(\boldsymbol{x})]-\mathbb{E}_{\boldsymbol{x} \sim p_{\theta}}[\nabla_{\theta}E_{\theta}(\boldsymbol{x})]
$$

これは，パラメータ$\theta$についての勾配．

分配関数$Z_\theta$は，$\theta$に依存する関数なので，$\theta$についての勾配を取ったときに，分配関数は残ってくる．

つまり，分配関数について考えない距離関数を考えれば良い．

そこでSBMでは，フィッシャーダイバージェンスを採用する．

フィッシャーダイバージェンス

$$
\frac{1}{2}\mathbb{E}_{p_{data}(\boldsymbol{x})}[||\nabla_{\boldsymbol{x}}\log p_{\theta}(\boldsymbol{x})-\nabla_{\boldsymbol{x}}\log p_{data}(\boldsymbol{x})||_2^2]
$$

みてすぐわかるとおり，フィッシャーダイバージェンスは$\boldsymbol{x}$についての勾配なので，$\theta$に依存しない．

つまり，勾配を取った時に分配関数について考えなくて良い．ということになる．

※ $\nabla_{\boldsymbol{x}} Z_{\theta}=0$

$$
\begin{align*}
\frac{1}{2}\mathbb{E}_{p_data(x)}[||s_{\theta}(x) - \nabla_x \log p_{data}(x)||_2^2] &= \frac{1}{2}\mathbb{E}_{p_{data(x)}}[(s_{\theta}(x) - \nabla_x \log p_{data(x)})^T(s_{\theta}(x) - \nabla_x \log p_{data(x)})]\\
&= \frac{1}{2}\mathbb{E}_{p_{data}(x)}[s_{\theta}(x)^T s_{\theta}(x) - s_{\theta}(x)^T \nabla_x \log p_{data}(x) - \nabla_x \log p_{data}(x)^T s_{\theta}(x) + \nabla_x \log p_{data}(x)^T \nabla_x \log p_{data}(x)]\\
&= \frac{1}{2}\mathbb{E}_{p_{data}(x)}[s_{\theta}(x)^T s_{\theta}(x) - s_{\theta}(x)^T \nabla_x \log p_{data}(x) - s_{\theta}(x)^T \nabla_x \log p_{data}(x) + \nabla_x \log p_{data}(x)^T \nabla_x \log p_{data}(x)]
\end{align*}
$$

内積は対象なので，$s_{\theta}(x)^T \nabla_x \log p_{data}(x) = \nabla_x \log p_{data}(x)^T s_{\theta}(x)$

ノルム表記に戻す

$$
= \frac{1}{2}\mathbb{E}_{p_{data}(x)}[||s_{\theta}(x)||_2^2 - s_{\theta}(x)^T \nabla_x \log p_{data}(x) - s_{\theta}(x)^T\nabla_x \log p_{data}(x) + ||\nabla_x \log p_{data}(x)||_2^2]
$$

以上のことを踏まえて式を整理する．
$$
\begin{align*}
&= \mathbb{E}_{p_{data}(x)}[||\frac{1}{2}s_{\theta}(x)||_2^2 - \frac{1}{2}2s_{\theta}(x)^T \nabla_x \log p_{data}(x) + \frac{1}{2}||\nabla_x \log p_{data}(x)||_2^2]\\
&= \mathbb{E}_{p_{data}(x)}[||\frac{1}{2}s_{\theta}(x)||_2^2 - s_{\theta}(x)^T \nabla_x \log p_{data}(x) + \frac{1}{2}||\nabla_x \log p_{data}(x)||_2^2]
\end{align*}
$$

また，期待値内の第2項 $s_{\theta}(x)^T \nabla_x \log p_{data}(x)$ に対して、次の恒等式が使えます。これは確率論と微分積分の結果で、スコア関数の性質を活用しています

$$
\mathbb{E}_{p_{data}(x)}[s_{\theta}(x)^T \nabla_x \log p_{data}(x)] = -\mathbb{E}_{p_{data}(x)}[tr(\nabla_x s_{\theta}(x))]
$$

これを使う．
$$
\mathbb{E}_{p_{data}(x)}[||\frac{1}{2}s_{\theta}(x)||_2^2 + tr(\nabla_x s_{\theta}(x)) + \frac{1}{2}||\nabla_x \log p_{data}(x)||_2^2]
$$

また，$\frac{1}{2}||\nabla_x \log p_{data}(x)||_2^2$はデータ分布に依存する値であり，$\theta$には依存しない値なので定数として扱える．したがって$C$とおける．

最終的に以下のようになる．

$$
\mathbb{E}_{p_{data}(x)}[||\frac{1}{2}s_{\theta}(x)||_2^2 + tr(\nabla_x s_{\theta}(x))] + C
$$

この式の第一項を，$\theta$について最適化することで生成モデルを学習できる．データ分布に依存する値を$C$としておいたのは，単純に最適化する時に定数は考えなくて良いからである．

$$
\mathbb{E}_{p_{data}(x)}[||\frac{1}{2}s_{\theta}(x)||_2^2 + tr(\nabla_x s_{\theta}(x))]
$$

意味もわからず変形したが，$tr(\nabla_x s_{\theta}(x))$はトレースと言う．

また，スコア関数は，元々一階微分されているものなので，トレースの中身は二階微分になっている．

つまり，ヘッセ行列のトレースを計算する必要があるということ．

しかし，スコア関数の勾配のトレース．

つまり対数尤度のヘッセ行列のトレースを計算するということ．この計算にはとてもコストがかかる．

スコアベースモデルでは，ここのヘッセ行列のトレースの計算をいかにして回避するかが，鍵になり．さまざまな手法が提案されている．